In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_fresh.pkl', 'df_ohlcs.pkl', 'df_symlots.pkl',
       'df_und_margins.pkl', 'df_unds.pkl', 'dfrq.pkl', 'qopts.pkl',
       'z_new_qopts_temp.pkl'], dtype='<U20')

# Engine to-do

In [ ]:
from engine import get_market

In [ ]:
MARKET = get_market()

In [ ]:
def get_market_action(MARKET: str) -> None:
    
    # Check if paper or live is needed
    paper_ip = input("\nShould Engine run on `Paper`? (Y/N) ").lower()
    if paper_ip == "y":
        RUN_ON_PAPER = True
    else:
        RUN_ON_PAPER = False
        
    # Check if existing pickles need to be deleted
    delete_ip = input("\n\nDelete Engine pickles? (Y/N) ").lower()
    if delete_ip == "y":
        DELETE_PICKLES = True
    else:
        DELETE_PICKLES = False
        
    # Check what needs to be done
    todo_dict = {0: 'quit', 1: f'all_{MARKET}', 2: f'{MARKET}_base', 
                 3: f'{MARKET}_symlots', 4: f'{MARKET}_ohlcs', 
                 5: f'{MARKET}_unds', 6: f'{MARKET}_chains', 
                 7: f'{MARKET}_qualopts', 8: f'{MARKET}_opt_prices',
                 9: f'{MARKET}_opt_margins'}
    
    todo_ip = f"What is to be done for {MARKET}? Choose from following numbers:\n"
    

    mktodo_range = list(range(len(todo_dict)))
    mktodo = f"What is to be done for {MARKET}? Choose from following numbers:\n"
    mktodo = mktodo + "\n0)  Quit this program!!!\n\n"
    mktodo = mktodo + "1)  Build from scratch. Run the entire Engine\n"
    mktodo = mktodo + "2)  Get only the base (df_symlots, df_unds, df_ohlcs, df_chains)\n"
    mktodo = mktodo + "3)  Get symbols with lots\n"
    mktodo = mktodo + "4)  Get OHLCs\n"
    mktodo = mktodo + "5)  Get underlyings\n"
    mktodo = mktodo + "6)  Get chains\n"
    mktodo = mktodo + "7)  Qualify options\n"
    mktodo = mktodo + "8)  Get all qualified option prices\n"
    mktodo = mktodo + "9)  Get all qualified option margins\n"

    mktodo = mktodo + '\n' + "-" * 70 + '\n'

    if MARKET == 'NSE':
        mktodo = mktodo + "10)  Build trades for Capstocks\n"

    mktodo = mktodo + '\n' + "=" * 70 + '\n'

    mktodo = mktodo + "\n11) Execute selected trades\n"
    mktodo = mktodo + "\n12) Delete all files\n"

    mktodo = mktodo + "\n"    
    
    

# Analysing fresh
## Build fresh

In [ ]:
# Ask and Run fresh
from fresh import get_fresh

THIS_FOLDER = ''
ip = input("Do you want to run Fresh (Y/N)?")
if ip.upper() == "Y":
    df_fresh = get_fresh(MARKET)
else:
    df_fresh = pd.read_pickle(DATAPATH.joinpath(THIS_FOLDER, 'df_fresh.pkl'))

In [ ]:
cols = ['symbol', 'strike', 'right', 'expiry', 'dte', 
        'margin', 'bid', 'ask', 'iv', 'und_iv','price', 'lot', 'comm', 'rom', 'sdMult', 'expRom', 'expPrice', 'qty']
df = df_fresh[df_fresh.price>0][cols].sort_values('rom', ascending=False)

In [ ]:
# How many nakeds have rom higher than 80% of expRom
df[df.rom >= df.expRom*0.8].sort_values('sdMult')